# Art Exploration Dashboard

This quick example programmatically buids a Weave Dashboard to explore and remix visual art.
* generate creative descriptions of artwork given some famous artists as reference points/inspiration
* create a Table (one column of text) of these art prompts (named "style_riff")
* use Stable Diffusion to generate art based on these prompts ("riff_art" column)
* inverse-generate a prompt which would lead to that image (and perhaps describe it better? "art_desc" column)
* imagine a perfect title for an artwork with that description ("desc title")
* finally, generate a remixed artwork for that title ("remixed_art"), except with more of the generated color specified in the horizontal panel (itself generated from the RGB values in the left sidebar)

## Play with these parameters

* the artist names and artist_id for which artist is the starting point
* the "palette" colors—these are the rows in the Dashboard
* the number of works (must be <= the number of colors in the palette and the number of artists
* the RGB values (0, 255)—these are combined to generate the "color" which is amplified in the remix art. Sometimes the color names are returned with extra words :)
* any of the prompts used to generate images or titles!


In [ ]:
import weave
from weave.legacy.weave import weave_internal
from weave.legacy.weave.ecosystem.all import langchain

In [ ]:
weave.legacy.weave.panels.Board(
    vars={
        "chat_model": langchain.chat_openai("gpt-3.5-turbo", 0.7),
        "artists" : ["René Magritte", "Josephine Wall", "Android Jones", "Alphonse Mucha", "Claude Monet"],
        "artist_id" : 0,
        "num_works": 3,
        "palette" : ["red", "orange", "yellow", "green", "blue", "violet"],
        "style_riff": 
            lambda chat_model, num_works, artists, palette, artist_id:
                weave.legacy.weave.ops.range(0, num_works, 1).map(
                    lambda row:
                        chat_model.predict(weave_internal.const("You are an expert art historian. In a sophisticated, avant-garde art museum, you see pieces in ")
                            + palette[row] + " by " + artists[artist_id]
                            + ", which has a fantastic contrast and synergy with the work of")
      
        ),
        "riff_art":
            lambda style_riff, num_works:
                weave.legacy.weave.ops.range(0, num_works, 1).map(
                    lambda row: style_riff[row]).stable_diffusion(),
        "art_desc":
            lambda riff_art, num_works:
                weave.legacy.weave.ops.range(0, num_works, 1).map(
                    lambda row: riff_art[row]).img2prompt(),
        "desc_title":
            lambda art_desc, num_works, chat_model:
                weave.legacy.weave.ops.range(0, num_works, 1).map(
                    lambda row: chat_model.predict(weave_internal.const("Art description: ")
                                + art_desc[row] + ". Perfect title:")),
        "R" : 168,
        "G" : 156,
        "B" : 234,
        "color" : lambda chat_model, R, G, B:
                        chat_model.predict(weave_internal.const("RGB value: (")
                        + R.toString() + "," + G.toString() + "," + B.toString()
                        + "). Precise color name:"),
        "remixed_art":
            lambda desc_title, num_works, color:
                weave.legacy.weave.ops.range(0, num_works, 1).map(
                    lambda row: weave_internal.const("a beautiful painting titled ")
                                 + desc_title[row] + " with more " + color).stable_diffusion()
                                                                    
    },
    panels=[
        weave.legacy.weave.panels.BoardPanel(
            lambda style_riff: weave.legacy.weave.panels.Table(style_riff, columns=[lambda col: col]),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=4, h=16)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda riff_art: weave.legacy.weave.panels.Table(riff_art, columns=[lambda col: col]),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=4, y=0, w=6, h=16)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda art_desc: weave.legacy.weave.panels.Table(art_desc, columns=[lambda col: col]),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=10, y=0, w=4, h=16)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda desc_title: weave.legacy.weave.panels.Table(desc_title, columns=[lambda col: col]),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=14, y=0, w=4, h=16)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda remixed_art: weave.legacy.weave.panels.Table(remixed_art, columns=[lambda col: col]),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=18, y=0, w=6, h=16)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda color: weave.legacy.weave.panels.StringEditor(color),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=7, y=16, w=10, h=4)
        ),
        
    ]
)